In [ ]:
# Install the libraries in your kernel
%pip install pandas
%pip install scikit-learn
%pip install xgboost
%pip install plotly


Note: you may need to restart the kernel to use updated packages.


In [2]:
# Make the necessary imports

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import plotly.express as px
#import chart_studio.plotly as py

In [3]:
# Read the (training data)

poems_df = pd.read_csv('poems.csv')
len(poems_df)

13854

In [4]:
poems_df['text'] = (poems_df['Poem'].str.replace('\n', ' ')).str.replace('\r', ' ')
poems_df.head()

,Unnamed: 0,Title,Poem,Poet,Tags,text
0,0,\r\n\r\n Objects Used to Pr...,"\r\n\r\nDog bone, stapler,\r\n\r\ncribbage boa...",Michelle Menting,NaN,"Dog bone, stapler, cribbage board, garl..."
1,1,\r\n\r\n The New Church\r\n...,\r\n\r\nThe old cupola glinted above the cloud...,Lucia Cherciu,NaN,"The old cupola glinted above the clouds, s..."
2,2,\r\n\r\n Look for Me\r\n\r\...,\r\n\r\nLook for me under the hood\r\n\r\nof t...,Ted Kooser,NaN,Look for me under the hood of that old ...
3,3,\r\n\r\n Wild Life\r\n\r\n ...,"\r\n\r\nBehind the silo, the Mother Rabbit\r\n...",Grace Cavalieri,NaN,"Behind the silo, the Mother Rabbit hunc..."
4,4,\r\n\r\n Umbrella\r\n\r\n ...,\r\n\r\nWhen I push your button\r\n\r\nyou fly...,Connie Wanek,NaN,When I push your button you fly off the...


In [5]:
N = 10

In [6]:
poems_df.groupby('Poet').count().sort_values(['Title'], ascending = False).reset_index().head(100).sample(10)

,Poet,Unnamed: 0,Title,Poem,Tags,text
16,William Blake,33,33,33,31,33
55,Mother Goose,23,23,23,23,23
24,Edgar Lee Masters,31,31,31,29,31
42,A. E. Stallings,25,25,25,22,25
41,Sir Thomas Wyatt,25,25,25,24,25
43,Naomi Shihab Nye,25,25,25,21,25
66,Wendy Videlock,21,21,21,20,21
87,Gerard Manley Hopkins,19,19,19,19,19
83,Carl Phillips,20,20,20,20,20
15,Samuel Menashe,33,33,33,30,33


In [30]:
top_poets = poems_df.groupby('Poet').count().sort_values(['Title'], ascending = False).reset_index().head(100)['Poet'].values


In [41]:
def experiment(state,N):
    top_poets = poems_df.groupby('Poet').count().sort_values(['Title'], ascending = False).reset_index().head(100).sample(N,random_state=state)['Poet'].values
    poems_sdf = poems_df[poems_df['Poet'].isin(top_poets)]
    
    vectorizer = TfidfVectorizer(max_features=5000,stop_words='english',min_df =.05,max_df=.95)
    vectors = vectorizer.fit_transform(poems_sdf['text']) 
    le = LabelEncoder()
    y_encoded = le.fit_transform(poems_sdf['Poet'])
    
    #data_dmatrix = xgb.DMatrix(data=vectors,label=y_encoded)
    #params = {'random_state':'42','num_class':N,'device':'gpu','max_depth':7,'eta':.1}
    
    #xgb_cv = xgb.cv(dtrain=data_dmatrix,params=params, nfold=5,metrics='merror')
    #print(xgb_cv)
    
    T = int(len(poems_sdf)*.8)
    model = xgb.XGBClassifier(random_state=42)
    model.fit(vectors[:T],y_encoded[:T])
    
    score = model.score(vectors[T:],y_encoded[T:])
    print(f'Accuracy round {state+1}: ',score)
    return(score)

In [43]:
total = 0

#number of experimental rounds
E = 5

#number of poets to include in each experiment
N = 10

for i in range(E):
    total+= experiment(i,N)

print('Average accuracy: ',total/E)

Accuracy round 1:  0.37254901960784315
Accuracy round 2:  0.21739130434782608
Accuracy round 3:  0.3225806451612903
Accuracy round 4:  0.46
Accuracy round 5:  0.5660377358490566
Average accuracy:  0.38771174099320327


In [44]:
# Translates the genres into numerical categories



In [12]:
# Specify our training data and training labels for xgboost



In [ ]:
# Running cross-validation for our XGB Classifier model gives a good idea of our model's accuracy

